In [283]:
import pandas as pd
import numpy as np
import mysql.connector

In [284]:
from sklearn import preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

dpd vs. ratio
<img src="dpd v. ratio.png">
total amount vs. ratio
<img src="total amount vs. ratio.png">
trained on 1000 examples
<img src="1k.png">
trained on 100k examples
<img src="100k.png">

In [285]:
hp_limit = 1000
train_limit = 10000
output = "ratio_total_per_balance"

df = pd.read_csv('data_invoce_topan.csv')

df["ratio_total_per_balance"] = df["cash_memo_balance_amount"] / df["cash_memo_total_amount"]
#df["ratio_total_per_balance"] =  df["cash_memo_total_amount"] - df["cash_memo_balance_amount"]

#plt.plot(df["DPD"], df["ratio_total_per_balance"], linestyle="None", marker='o')
#plt.plot(df["cash_memo_total_amount"], df["ratio_total_per_balance"], linestyle="None", marker='o')
#plt.show()

df = df.drop("cash_memo_total_amount", axis=1)

df_part = df[:hp_limit]

print(df_part)

train_set, test_set = train_test_split(df_part, test_size=0.2)

train_labels = train_set[output].copy()
train_features = train_set.drop(output, axis=1)

test_labels = test_set[output].copy()
test_features = test_set.drop(output, axis=1)



FileNotFoundError: [Errno 2] File b'data_invoce_topan.csv' does not exist: b'data_invoce_topan.csv'

In [ ]:
"""from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

class DataFrameSelector(BaseEstimator, TransformerMixin): 
    def __init__(self, attibute_names):
        self.attibute_names = attibute_names
    def fit(self, X, y=None):
        return(self)
    def transform(self, X): 
        return(X[self.attibute_names].values)

#%run CategoricalEncoder.py

cat_attributes = ['outlet_code']
num_attributes = ['DPD','product_price','ratio_total_per_balance']

num_pipeline = Pipeline([
    ('selector',DataFrameSelector(num_attributes)),
#     ('imputer',Imputer(strategy="median")),
    ('std_scaler', StandardScaler()),])

cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attributes)),
    ('label_binarizer',CategoricalEncoder(encoding="onehot-dense")),])

full_pipeline = FeatureUnion(transformer_list=[
    ('cat_pipeline',cat_pipeline),
    ('num_pipeline',num_pipeline),])

df_prepared = full_pipeline.fit_transform(train_set.copy())
df_labels = train_set[output].copy()
df_prepared.shape"""

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
def evaluate(labels, predictions):
    #Begin rms error
    rmse_accuracy = 100 - np.sqrt(mean_squared_error(labels, predictions))/(np.amax(labels) - np.amin(labels)) * 100

    errors = abs(predictions - labels) # this was one way I found in "towards data science" website, we can use the NRMSD 
                                        # to calculate the error. 
    mape = 100 * (errors / test_labels)  # Calculate mean absolute percentage error (MAPE)
    #print(mape)
    mape_accuracy = 100 - np.mean(mape) # Calculate and display accuracy
    plt.plot(labels, predictions)
    plt.show()
    accuracy_grid = {"RMSE Percentage Accuracy": rmse_accuracy, "r2 score /1":r2_score(labels, predictions)}
    return accuracy_grid

In [ ]:
"""from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(df_prepared, df_labels)

df_test_labels = test_set[output]

df_test = full_pipeline.transform(test_set)

predictions = lin_reg.predict(df_test)

print(str(evaluate(df_test_labels, predictions)) + ' normalized percent error')
compare = {'predicted':list(predictions),'examples':list(df_test_labels)}
pd.DataFrame(compare)
"""


In [ ]:
"""https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74 """

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

def RandomCVTuningRandomForest(train_features, train_labels):
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 600, stop = 1600, num = 3)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 30, num = 3)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestRegressor()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    rf_random.fit(train_features, train_labels)
    print(rf_random.best_params_)
    return rf_random

hp_result = RandomCVTuningRandomForest(train_features, train_labels)


In [ ]:
#print(Predictor.predict(test_features))
df_full = df[hp_limit:hp_limit+train_limit]
train_set, test_set = train_test_split(df_full, test_size=0.2)

train_labels = train_set[output].copy()
train_features = train_set.drop(output, axis=1)

test_labels = test_set[output].copy()
test_features = test_set.drop(output, axis=1)

Predictor = hp_result.best_estimator_

print(type(train_labels), type(train_features))

Predictor.fit(train_features, train_labels)

print(evaluate(test_labels, Predictor.predict(test_features)))


In [ ]:
import ipywidgets as widgets
from IPython.display import display, Image
from ipywidgets import HBox, Label
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


# w = widgets.IntSlider()
# display(w)
IntSliderlayout = widgets.Layout(width='70%', height='50px') # the layout that contains the intSlider

outlet_code_Textwidget = widgets.Text(
#     description = 'outlet_code',
#     layout=widgets.Layout(width='50%', height='80px')
)
outletBox = HBox([Label('outlet code'), outlet_code_Textwidget]) # a box tha holds the outlet code text box. This is used with this widget
                                                     # and other widgets to allow for more room for the label as description
                                                     # doesn't scale properly in size.
cash_memo_balance_amount_Textwidget = widgets.Text(
#     description = 'cash_memo_balance_amount'
)
cash_memo_balance_amountBox = HBox([Label('cash memo balance amount'), cash_memo_balance_amount_Textwidget])

# cash_memo_total_amount_Textwidget = widgets.Text()
# cash_memo_total_amountBox = HBox([Label('cash memo total amount'), cash_memo_total_amount_Textwidget])

product_price_Textwidget = widgets.Text(
#     description = 'product_price'
)
product_priceBox = HBox([Label('product price'), product_price_Textwidget])

quantity_Textwidget = widgets.Text(
#     description = 'quantity'
)
quantityBox = HBox([Label('quantity'), quantity_Textwidget])

DPD_IntSliderwidget = widgets.IntSlider(
    value = 0, # the initial value
    min = 0,  # setting the min and max values of the slider
    max = 30,    
#     description='DPD:', # a text describing the slider
    disabled=False, # if you want to disable the slider or enable it.
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout = IntSliderlayout,
    readout_format='d'
)
DPDBox = HBox([Label('DPD (max value = 30, min value = 0)'), DPD_IntSliderwidget])

submit_button = widgets.Button(
    description = 'submit features'
)

ratiopaidLabel = Label(
)

# defining and setting up the image widget
imgFile = open("logo.png", "rb")
image = imgFile.read()
imageWidget = widgets.Image(
    value=image,
    format='png',
    width=500,
    height=110,
)

# the method that used in the submit button click event
def submitFeatures(btn_object):
    
    outlet_code = int(outlet_code_Textwidget.value)
    cash_memo_balance = float(cash_memo_balance_amount_Textwidget.value)
    productprice = float(product_price_Textwidget.value)
    quantity = int(quantity_Textwidget.value)
    dpd = int(DPD_IntSliderwidget.value)
    
    predictor_input = [[outlet_code,
                               cash_memo_balance,
                                productprice,
                                quantity,
                                dpd
                               ]]
    
    print(predictor_input)
    
    prediction = Predictor.predict(predictor_input)
    prediction = str(prediction[0])
    
    print('outlet: ', outlet_code_Textwidget.value)
    print('amountDue: ', cash_memo_balance_amount_Textwidget.value)
    print('productPrice: ', product_price_Textwidget.value)
    print('DPD: ', DPD_IntSliderwidget.value)
    print('quantity: ', quantity_Textwidget.value)
    print('cash_memo_total_amount: ', cash_memo_total_amount_Textwidget.value)
    print('ratio: ', prediction)
    
    ratiopaidLabel.description = prediction
    display( Label(prediction) )
    
  # storing the result of the final quantity value here
#     print('-- prediction of ratio paid ', prediction)

display(imageWidget) # showing the image 
display(outletBox) # showing the outlet text field widget
display(cash_memo_balance_amountBox) # showing the cash_memo_balance text field widget
display(product_priceBox) #  showing the product_price text field widget
display(DPDBox) #  showing the DPD slider widget
# display(cash_memo_total_amountBox) #  showing the cash_memo_total text field widget
display(quantityBox)
display(submit_button) #  showing the submit button widget
submit_button.on_click(submitFeatures)

display(Label('the Paid Ratio based on the features is: '))  # showing the quantity label 

display(ratiopaidLabel) # showing the quantity label result 
# w.close()